In [2]:
import cv2
import numpy as np

def dark_channel_prior(image, patch_size=15, omega=0.95):
    """
    应用暗通道先验抑制强光。
    
    :param image: 输入图像（BGR）
    :param patch_size: 局部patch大小
    :param omega: 控制透射率的强度
    :return: 处理后的图像
    """
    image = image.astype(np.float32) / 255.0
    min_channel = np.min(image, axis=2)
    
    # 计算暗通道
    dark_channel = cv2.erode(min_channel, np.ones((patch_size, patch_size)))
    
    # 估计大气光
    num_pixels = image.shape[0] * image.shape[1]
    flat_dark = dark_channel.flatten()
    indices = np.argsort(flat_dark)[-int(0.001 * num_pixels):]
    atmospheric_light = np.max(image.reshape(-1, 3)[indices], axis=0)
    
    # 计算透射率
    transmission = 1 - omega * dark_channel
    
    # 恢复图像
    result = np.zeros_like(image)
    for i in range(3):
        result[:, :, i] = (image[:, :, i] - atmospheric_light[i]) / np.maximum(transmission, 0.1) + atmospheric_light[i]
    
    result = np.clip(result * 255, 0, 255).astype(np.uint8)
    return result

# 读取视频流
cap = cv2.VideoCapture('highway_n.avi')  # 替换为你的视频路径

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 应用暗通道先验
    processed_frame = dark_channel_prior(frame)
    
    # 显示原始和处理后的帧
    cv2.imshow('Original', frame)
    cv2.imshow('Dark Channel Processed', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()